In [1]:
# import pymysql
# from sqlalchemy import create_engine
# import sys
# from abc import ABC, abstractmethod
# from typing import *

import re

from prettytable import PrettyTable
import pandas as pd
pd.options.display.float_format = '{:.30f}'.format
import pprint

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 0. 모듈의 상위 패키지를 파이썬 경로로 설정

In [2]:
import sys

In [3]:
sys.path.append("C:\Pseudonymizer\pseudonymizer")
# /path/to/your/module

#### 1. MySQL Server 내 데이터베이스 테이블을 Python 스크립트와 연동하여 데이터 로드

In [4]:
from pseudonymizer.encryptionPseudonyms.pyMySQLQuery import PyMySQLQuery, ConnectMySQLserver

In [5]:
# TEST CODE
queryObject = PyMySQLQuery(pw = "0123")

In [6]:
queryObject.connectDatabase(
    serverIP = "localhost", 
    port_num = 3306, 
    user_name = "root", 
    database_name = "FINANCIALCONSUMER", 
    kr_encoder = "utf8")

In [7]:
SQL = input("SQL 쿼리문 입력변수 = ")
    # SELECT * FROM DATA_FINANCE

In [8]:
queryObject.dataQueryLanguage(sql = SQL)

In [ ]:
# DATA_FINANCE_TUPLE = queryObject.executeQuery()
    # AttributeError: 'NoneType' object has no attribute 'execute'

In [10]:
DATA_FINANCE_TABLE = queryObject.executeQueryAsDataFrame()

In [11]:
type(DATA_FINANCE_TABLE)
DATA_FINANCE_TABLE.info()
DATA_FINANCE_TABLE.tail(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   NUM_SERIAL                  95000 non-null  int64 
 1   NAME                        95000 non-null  object
 2   GENDER                      95000 non-null  object
 3   AGE                         95000 non-null  int64 
 4   PHONE_NUMBER                95000 non-null  object
 5   ZIP_CODE                    95000 non-null  int64 
 6   HOME_ADDRESS                95000 non-null  object
 7   HOME_TYPE                   95000 non-null  object
 8   INCOME_BRACKET              95000 non-null  int64 
 9   CREDIT_SCORE                95000 non-null  int64 
 10  REPAYMENT_RISK_INDEX        95000 non-null  int64 
 11  AMT_CREDITCARD_PAYMENT      95000 non-null  int64 
 12  AMT_CASHADVANCE_PAYMENT     95000 non-null  int64 
 13  NUM_CREDITCARD_ISSUANCES    95000 non-null  in

,NUM_SERIAL,NAME,GENDER,AGE,PHONE_NUMBER,ZIP_CODE,HOME_ADDRESS,HOME_TYPE,INCOME_BRACKET,CREDIT_SCORE,REPAYMENT_RISK_INDEX,AMT_CREDITCARD_PAYMENT,AMT_CASHADVANCE_PAYMENT,NUM_CREDITCARD_ISSUANCES,NUM_CREDITCARD_CANCELED,TF_LOAN,AMT_CREDITLOAN,AMT_CREDITLOAN_OUTSTANDING,TF_PENSION,AMT_PENSION
94997,94997,명시라,female,66,010-2536-2511,14614,경기도 부천시 장말로 292(심곡동),연립주택,4,294,26,8488084,7377056,8,2,N,0,0,N,0
94998,94998,롤미라,female,73,010-5091-5053,32755,충청남도 금산군 부리면 물페기길 66,오피스텔,6,986,7,5109440,3625570,3,2,N,0,0,N,0
94999,94999,안전웅,male,46,010-9622-1090,10585,경기도 고양시 덕양구 안진6길 12(지축동),오피스텔,7,44,76,874939,5785351,7,0,N,0,0,N,0


#### 2. 가명처리 절차 : 개인정보가 포함된 데이터를 비식별조치를 통해 가명정보로 변환

In [ ]:
from pseudonymizer.pseudonymizer import Pseudonymizer

In [ ]:
from pseudonymizer.pseudonym import Pseudonym
# ImportError: cannot import name 'Pseudonymizer' from 'pseudonymizer' (unknown location)